## Tacotron 2 inference code 
Edit the variables **checkpoint_path** and **text** to match yours and run the entire code to generate plots of mel outputs, alignments and audio synthesis from the generated mel-spectrogram using Griffin-Lim.

#### Import libraries and setup matplotlib

In [1]:
import matplotlib
%matplotlib inline
import matplotlib.pylab as plt
import IPython.display as ipd


import sys
sys.path.append('hifigan/')
import numpy as np
import torch
import librosa
import librosa.display
import math
import json
import os
import time
import soundfile as sf

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT, STFT
from audio_processing import griffin_lim
from train import load_model
from text import text_to_sequence, symbol_to_id
from text.cleaners import transliteration_ua_cleaners, english_cleaners, transliteration_cleaners,transliteration_cleaners_with_stress
from text.rudict import RuDict

import torch.nn.utils.prune as prune
from torch.nn import functional as F

from sklearn.metrics.pairwise import cosine_similarity as cs


from hifigan.meldataset import MAX_WAV_VALUE
from hifigan.models import Generator
from hifigan.env import AttrDict

from audio_processing import get_mel

loaded = False



In [2]:
def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='lower', 
                       interpolation='none')
    plt.savefig('out.png')
    ipd.display(ipd.Image('out.png'))

In [3]:
hparams = create_hparams()
hparams.sampling_rate = 22050
# hparams.end_symbols_ids = [symbol_to_id[s] for s in '?!.']
# hparams.use_gst = False

In [4]:
checkpoint_path = "outdir/checkpoint_2500"
model = Tacotron2(hparams).cuda()
model.load_state_dict(torch.load(checkpoint_path)['state_dict']) #,map_location=torch.device('cpu')
_ = model.cuda().eval().half()

In [15]:
ref_mel = get_mel('test/me1.wav', hparams).type(torch.HalfTensor).cuda()
print(ref_mel.dtype, ref_mel.shape)

torch.float16 torch.Size([1, 80, 872])


In [6]:
print(ref_mel)

tensor([[[-6.5586, -6.9414, -6.6484,  ..., -5.7969, -4.7070, -3.5684],
         [-6.6914, -6.9883, -6.5039,  ..., -5.3203, -4.1797, -3.1133],
         [-6.5273, -7.0859, -7.3281,  ..., -2.0859, -1.9082, -2.1719],
         ...,
         [-4.7383, -4.3242, -3.8496,  ..., -7.1875, -7.8164, -7.9922],
         [-4.6680, -4.4102, -3.7910,  ..., -7.1836, -7.6211, -8.4766],
         [-4.7852, -4.5273, -4.2148,  ..., -7.7656, -8.1797, -8.6328]]],
       device='cuda:0', dtype=torch.float16)


In [7]:

text = "Тетрагидропиранилциклопентилтетрагидропиридопиридиновые вещества".lower()
text = "Мин+истр здравоохран+ения улет+ел. И обещ+ал верн+уться. С вакц+инами. Нав+ерно, +есть как+ой-то м+ир, где мин+истрам л+ично н+адо лет+ать за вакц+инами с четырьм+я перес+адками во вр+емя эпид+емии. М+ожет по+этому в Зимб+абве уже вакцин+ируют, потом+у чт+о их мин+истр л+ично с+ел на слон+а и по+ехал на нем в больш+ой г+ород преклон+ить кол+ено и получ+ить вакц+ину? Так+ой м+ир нам пыт+аются прод+ать?".lower()
# text = "Вс+е смеш+алось в д+оме Обл+онских. Жен+а узн+ала, что муж был в св+язи с б+ывшею в их д+оме франц+уженкою-гуверн+анткой, и объяв+ила м+ужу, что не м+ожет ж+ить с ним в одн+ом д+оме. Полож+ение это продолж+алось уже третий д+ень и муч+ительно ч+увствовалось и сам+ими супр+угами, и вс+еми чл+енами семь+и, и домоч+адцами."
# text = "Тв+орог или твор+ог, к+озлы или козл+ы, з+амок или зам+ок.".lower()
# text="мн+е хот+елось б+ы? сказ+ать к+ак я призн+ателен вс+ем прис+утсвующим сд+есь."
# text = "tak+oi m+ir n+am pyt+aiutsia prod+at'?"
# text = "Как пройти в дом?"
# text = "В+ыйду,,,,,,, н+очью,,, в п+оле,,, с кон+ем"
text = "Н+очь, +улица, фон+арь, апт+ека. Бессм+ысленный, и т+усклый св+ет. Жив+и ещ+е х+оть ч+етверть в+ека - Вс+ё б+удет т+ак. Исх+ода н+ет."
# text = "Ночь, улица, фонарь, аптека. Бессмысленный, и тусклый свет. Жив+и еще хоть четверть века - Все будет так. Исхода нет."
# text = "Игра - тип осмысленной непродуктивной деятельности, где мотив лежит не в её результате, а в самом процессе. Также термин игра используют для обозначения набора предметов или программ, предназначенных для подобной деятельности."
# text = "Прижм+ись к+о мн+е кр+епче и бл+иже. Не ж+ил я,, блужд+ал ср+едь чуж+их. О с+он м+ой,, Я н+овое в+ижу, В бред+у поцел+уев тво+их!"

# text = "Тогд+а почем+у ж+е я ощущ+аю себ+я н+а гр+ани физ+ической катастр+офы?"
# text ="Ты согласен?"
# text = "Здр+аствуйте, мен+я зов+ут Андр+ей Никиш+аев, я с+иньер архит+ектор в комп+ании манив+ео. П+о мо+им д+анным у в+ас не закр+ытый кред+ит на с+умму в+осемдесят тр+и т+ысячи тр+иста шестдес+ят тр+и гр+ивны. Н+ужно погас+ить!"
# text = "С к+аждым дн+ем весн+ы стан+овится вс+е тепл+ее, а +это р+адость для к+аждого вод+ителя."
# text = "Молод+ой парн+ишка Т+анг С+ан одн+ажды оступ+ился и сл+едуя сво+им жел+аниям и пр+ихотям вор+ует секр+етные уч+ения в своей школе боевых искусств."

print(transliteration_cleaners_with_stress(text))
sequence = np.array(text_to_sequence(text, ['transliteration_cleaners_with_stress']))[None, :]
print(sequence)
sequence = torch.autograd.Variable(
    torch.from_numpy(sequence)).cuda().long()

Sn+och', +ulitsa, fon+ar', apt+eka. bessm+yslennyi, i t+usklyi sv+et. zhiv+i eshch+e kh+ot' ch+etvert' v+eka - vs+io b+udet t+ak. iskh+oda n+et.E
[[12 27 40 28 16 21  3  6 11 40 34 25 22 33 32 14  6 11 19 28 27 40 14 31
   3  6 11 14 29 33 40 18 24 14  7 11 15 18 32 32 26 40 38 32 25 18 27 27
  38 22  6 11 22 11 33 40 34 32 24 25 38 22 11 32 35 40 18 33  7 11 39 21
  22 35 40 22 11 18 32 21 16 21 40 18 11 24 21 40 28 33  3 11 16 21 40 18
  33 35 18 31 33  3 11 35 40 18 24 14 11  1 11 35 32 40 22 28 11 15 40 34
  17 18 33 11 33 40 14 24  7 11 22 32 24 21 40 28 17 14 11 27 40 18 33  7
  13]]


In [8]:
def model_size(model):
    torch.save(model.state_dict(), "temp.p")
    size = ('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')
    return size

In [9]:


# bit8_model = torch.quantization.quantize_dynamic(
#         model, {torch.nn.LSTM, torch.nn.Linear, torch.nn.LSTMCell, torch.nn.GRUCell}, dtype=torch.qint8
# )

# print(model_size(bit8_model))
# for name, module in bit8_model.named_modules():
#     if isinstance(module, torch.nn.Conv1d):
#         prune.l1_unstructured(module, name='weight', amount=0.2)
#     elif isinstance(module, torch.nn.Linear):
#         prune.l1_unstructured(module, name='weight', amount=0.2)
   
# print(model_size(bit8_model))

In [ ]:
%%time
# np.random.seed(16)

z_scale = np.ones(32, dtype=np.float32)
z_scale[7] = 1.0

_, mel_outputs, mel_outputs_postnet, _, alignments, emb2 = model.inference(sequence,seed=None,scale=1.0, token_idx=)

In [ ]:
plot_data((mel_outputs.float().data.cpu().numpy()[0],
           mel_outputs_postnet.float().data.cpu().numpy()[0],
           alignments.float().data.cpu().numpy()[0].T))

In [ ]:
emb2

In [ ]:
emb2

In [ ]:
loaded = False
def load_checkpoint(filepath, device):
    assert os.path.isfile(filepath)
    print("Loading '{}'".format(filepath))
    checkpoint_dict = torch.load(filepath, map_location=device)
    print("Complete.")
    return checkpoint_dict

if not loaded:
    device = torch.device('cuda')
    with open('hifigan/config.json') as fp:
        json_config = json.load(fp)
        h = AttrDict(json_config)
    generator = Generator(h).to(device)

    state_dict_g = load_checkpoint("hifigan/g_02500000", device)
    generator.load_state_dict(state_dict_g['generator'])
    generator.eval()
    generator.remove_weight_norm()
    
#     generator = torch.quantization.quantize_dynamic(
#         generator, {torch.nn.LSTM, torch.nn.Linear, torch.nn.LSTMCell, torch.nn.GRUCell}, dtype=torch.qint8
#     )
    loaded=True

def inference(mel, generator):
    

    
    with torch.no_grad():
        y_g_hat = generator(mel)
        audio = y_g_hat.squeeze()
        audio = audio * MAX_WAV_VALUE
        audio = audio.detach().cpu().numpy()#.astype('int16')

        return audio
    


In [ ]:
s = time.perf_counter()
mel = mel_outputs_postnet.type(torch.float32)
print(time.perf_counter()-s)
            

In [ ]:
%%time
audio = inference(mel, generator)
ipd.Audio(audio, rate=hparams.sampling_rate)  

In [ ]:
for i in range(32):
    z_scale = np.ones(32, dtype=np.float32)
    z_scale[10] = 1.5

    _, mel_outputs, mel_outputs_postnet, _, alignments, emb2 = model.inference(sequence,seed=1234,z_scale=z_scale, reference_mel=ref_mel)
    mel = mel_outputs_postnet.type(torch.float32)
    audio = inference(mel, generator)
    sf.write('res/z_%s.wav' % i, audio*0.0002, 22050)

In [ ]:
norm = F.normalize(vect, dim=1).detach().cpu().numpy()
res =  cs(norm, norm)
res

In [ ]:
(emb1.detach().cpu().numpy()[0][0] - emb2.detach().cpu().numpy()[0][0])

In [ ]:
print(emb1.detach().cpu().numpy().shape)